In [ ]:
#convolution testing 

function conv_kernel(x, c, δ, period)
    f = (y)  -> 1 / (δ * sqrt(2 * π)) * exp(-(y-c)^2 / (2 * δ^2))
    f_norm, error = quadgk(f, c-period/2, c+period/2)
    f_normalized = f(x)/f_norm
    return f_normalized
end

function extended_conv_kernel(x, c, δ, x_l, x_r)
    f = (y)  ->  exp(-(y-c)^2 / (2 * δ^2)) # 1 / (δ * sqrt(2 * π)) 
    K=1000
    L=x_r-x_l
    s=0.0
    for k in -K:K    
        s += f(x+k*L)
#        f_norm, error = quadgk(f, c-period/2, c+period/2)
#        f_normalized = f(x)/f_norm
    end
    return s
end

function normalized_ext_conv_kernel(x_grid,x_l, x_r, c, δ)
    vals = [extended_conv_kernel(x, c, δ, x_l, x_r) for x in x_grid]
    dx = x_grid[2] - x_grid[1]
    integral = sum(vals) * dx
    return vals ./ integral
end



function m_T_func(x, x_l, x_r)
    m_T_func_unnorm = (x) -> exp(-(x - 0.5)^2 / (0.1)^2)
    oneOverC, error = quadgk(m_T_func_unnorm, x_l, x_r) #Why 0, 1? Double check
    return (1 / oneOverC) * m_T_func_unnorm(x)
end



function test_old_conv_func(m_vec, x_vec, δ, c)
    ### Double check if this is actually correct, not completely sure...
    ### The "physical domain" is [0,1), but we extend to (-1,2) to avoid boundary artifacts...
    ### What is then the correct way to handle the convolution?
    old_ϕ_δ = (x, δ) -> 1 / (δ * sqrt(2 * π)) * exp(-(x-c)^2 / (2 * δ^2))
    N_h = length(m_vec)
    h = x_vec[2] - x_vec[1]
    conv_vec = Vector{Float64}(undef, N_h)

    # Base offsets from reference node and minimum-image wrap
    # Δs = x_vec .- x_vec[1]
    # Δs .-= round.(Δs)  # map to (-0.5, 0.5]
    Δs = (x_vec .- x_vec[1]) .- round.(x_vec .- x_vec[1])  # (-0.5, 0.5] 

    # Periodic kernel weights and discrete normalization
    weights = old_ϕ_δ.(Δs, δ)
    Z = h * sum(weights)
    weights ./= Z

    # Circular convolution with circulant weights (O(N^2), robust and simple)
    for j in 1:N_h
        s = 0.0
        for i in 1:N_h
            idx = mod(j - i, N_h) + 1  # 1-based index
            s += weights[idx] * m_vec[i]
        end
        conv_vec[j] = h * s
    end
    return conv_vec
end




#= function conv_kernel_per(x, c, δ, torus_l, torus_r)
    period = torus_r-torus_l
    
    #x_on_torus = torus_l + mod(x_grid-torus_l, L)
    f = (y) -> 1 / (δ * sqrt(2 * π)) * exp(-( (torus_l + mod(y-torus_l, period)) -c)^2 / (2 * δ^2))
    f_norm, error = quadgk(f, torus_l, torus_r)
    f_normalized = f(x)/f_norm
    return f_normalized
end =#

h = 1 / 2^9
x_l = -1
x_r = 2
x_grid = x_l:h:(x_r-h)
δ = 0.2
c = x_l #(x_l+x_r)/2
using FFTW

function test_func(x)
    return  1.5^2-(x-0.5)^2
end




conv_kernel_vec = conv_kernel.(x_grid, c,δ, x_r-x_l)
m_T_vec = m_T_func.(x_grid, x_l,x_r)
test_func_vec = test_func.(x_grid)

conv_vec = zeros(length(m_T_vec))
for j in 1:length(x_grid)
    temp_conv=0 
    for k in 1:length(x_grid)
        idx = mod((j-1)-(k-1), length(x_grid))+1
        temp_conv += conv_kernel_vec[k]*test_func_vec[idx]
    end
    conv_vec[j]=temp_conv
end
conv_vec = conv_vec .* (x_r-x_l)/length(x_grid)


conv3 = test_old_conv_func(test_func_vec, x_grid, δ, c)

conv4 = normalized_ext_conv_kernel(x_grid,x_l, x_r, c, δ)

Ff = fft(conv4)
Gg = fft(test_func_vec)
conv_coeffs = Ff .* Gg
conv2= real(ifft(conv_coeffs))
conv2 = conv2 .* (x_r-x_l)/length(conv4)

function fft_conv(x_grid, vec_to_convolve, x_l, x_r, δ)
    c=x_l
    kernel = normalized_ext_conv_kernel(x_grid, x_l, x_r, c, δ)
    Ff = fft(kernel)
    Gg = fft(vec_to_convolve)
    conv_coeffs = Ff .* Gg
    convolution_vec = real(ifft(conv_coeffs))
    return convolution_vec .* (x_r-x_l)/length(kernel)
end



p1 = plot(x_grid, conv_kernel_vec)
p6 = plot(x_grid, conv4)
#plot(x_grid, conv_kernel_per.(x_grid, 0.0, 0.1, x_l, x_r))
#p2 = plot(x_grid, m_T_func.(x_grid, x_l,x_r))
p2b = plot(x_grid, test_func.(x_grid))
#p3 = plot(x_grid, conv_vec)
p4 = plot(x_grid, conv2)
#p5 =plot(x_grid, conv3)
display(p1)
display(p6)
#display(p2)
display(p2b)
#display(p3)
display(p4)
#display(p5)
